In [1]:
#Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Conv1D, MaxPooling1D, Dropout, Flatten, RepeatVector, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
import tensorflow_addons as tfa

2023-12-17 16:11:08.829284: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-17 16:11:12.666225: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 16:11:12.666312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 16:11:13.552015: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 16:11:15.702517: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-17 16:11:15.718196: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# Load the dataset
data = pd.read_csv("/home/disi/ff/Datasets/Dataset_1/forcasting.csv")

In [3]:
data['created_at'] = pd.to_datetime(data['created_at'])
data = data.sort_values(by='created_at')
forecast_cols = ['humidity (%)', 'temperature (DegCel)', 'light intensity']
df_forecast = data[forecast_cols]
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_forecast)

In [4]:
def create_sequences(data, input_seq_length, output_seq_length):
    X, y = [], []
    for i in range(len(data) - input_seq_length - output_seq_length + 1):
        seq_x = data[i:i + input_seq_length]
        seq_y = data[i + input_seq_length:i + input_seq_length + output_seq_length]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

input_seq_length = 10
features = 3
output_seq_length = 5
X, y = create_sequences(scaled_data, input_seq_length, output_seq_length)
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

K.clear_session()

input_layer = Input(shape=(input_seq_length, features))

In [9]:
# Convolutional layers with BatchNormalization and regularization
x = Conv1D(16, (2), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(input_layer)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=(2))(x)
x = Conv1D(32, (2), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(x)
x = BatchNormalization()(x)
x = Conv1D(64, (2), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = RepeatVector(output_seq_length)(x)

# LSTM layers with dropout
x = LSTM(100, activation='relu', dropout=0.2, return_sequences=True, kernel_regularizer=regularizers.l2(0.001))(x)
x = LSTM(100, activation='relu', dropout=0.2, return_sequences=True, kernel_regularizer=regularizers.l2(0.001))(x)
output = TimeDistributed(Dense(3, activation = 'sigmoid'))(x)

model = Model(inputs=input_layer, outputs=output)

# Optimizer with custom learning rate
#optimizer = Adam(learning_rate=0.0001, weight_decay=1e-5)
optimizer = tfa.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.000001)

# Compile model with regularization loss
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

# Early stopping and learning rate reduction on plateau
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 3)]           0         
                                                                 
 conv1d_12 (Conv1D)          (None, 10, 16)            112       
                                                                 
 batch_normalization_8 (Bat  (None, 10, 16)            64        
 chNormalization)                                                
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 5, 16)             0         
 g1D)                                                            
                                                                 
 conv1d_13 (Conv1D)          (None, 5, 32)             1056      
                                                                 
 batch_normalization_9 (Bat  (None, 5, 32)             128 

In [10]:
# Implement early stopping
#early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), verbose=1) #callbacks=[early_stopping])


Epoch 1/50
95/95 [==============================] - 6s 19ms/step - loss: 0.6232 - accuracy: 0.8549 - val_loss: 0.5559 - val_accuracy: 0.7143
Epoch 2/50
95/95 [==============================] - 1s 14ms/step - loss: 0.4474 - accuracy: 0.9420 - val_loss: 0.4313 - val_accuracy: 0.9654
Epoch 3/50
95/95 [==============================] - 1s 13ms/step - loss: 0.3497 - accuracy: 0.9514 - val_loss: 0.3291 - val_accuracy: 0.9336
Epoch 4/50
95/95 [==============================] - 1s 13ms/step - loss: 0.2761 - accuracy: 0.9550 - val_loss: 0.2533 - val_accuracy: 0.9231
Epoch 5/50
95/95 [==============================] - 1s 13ms/step - loss: 0.2191 - accuracy: 0.9563 - val_loss: 0.1984 - val_accuracy: 0.9223
Epoch 6/50
95/95 [==============================] - 1s 14ms/step - loss: 0.1754 - accuracy: 0.9543 - val_loss: 0.1584 - val_accuracy: 0.9246
Epoch 7/50
95/95 [==============================] - 1s 13ms/step - loss: 0.1420 - accuracy: 0.9543 - val_loss: 0.1282 - val_accuracy: 0.9445
Epoch 8/50
95

In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, precision_score, recall_score, f1_score
import numpy as np

# Make predictions
y_pred = model.predict(X_test)

# Flatten the predictions and true values for simplicity
y_true_flat = y_test.flatten()
y_pred_flat = y_pred.flatten()

# Calculate MAE, RMSE, and MAPE
mae = mean_absolute_error(y_true_flat, y_pred_flat)
rmse = np.sqrt(mean_squared_error(y_true_flat, y_pred_flat))
mape = np.mean(np.abs((y_true_flat - y_pred_flat) / y_true_flat)) * 100

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape}%")

48/48 [==============================] - 1s 3ms/step
MAE: 0.03861124913605602
RMSE: 0.052105525110847005
MAPE: 27.51070814087261%
